# IPM DSS

## import modules

In [ ]:
import pandas

from openalea.dss import Hub, Model

from collections import UserDict

## Access to DSS catalog

In [102]:
#hub = Hub()
#hub.list_dss(ViewDataFrame=True)
#print(hub) 

#hub.display(ViewDataFrame=False) #to improve
import pandas
from agroservices import IPM
import json
import xarray as xr
import matplotlib.pyplot as plt

class Hub:
    """Class Hub
    
    Hub allows to access IPM catalog and get one model. 
    """
    
    def __init__(self):
        """[summary]
        """
        self.ipm_hub=IPM()
        self._catalog = None
    
    @property 
    def catalog(self):
        """ Trasform ipm catalogue from agroservices request in dict

        Returns
        -------
        [dict]
            [dict of dss catalog with meta-information by dss and model]
        """
        if self._catalog is None:
            self._catalog = self.ipm_hub.get_dss()
            
        return  {el["id"]: {item["id"]:item for item in el["models"]} for el in self._catalog}
    
    
    def display(self,view="dataframe"):
        """Display catalog meta information (dss, model and description)

        Parameters
        ----------
        view : str, optional
            [choose the type of catalog visualisation dataframe or dict], by default "dataframe"

        Returns
        -------
        [dataframe or dict]
            [return a dataframe or a dict]
        """
        if view=="dataframe":
            df=pandas.Series(self.catalog).apply(pandas.Series).stack().apply(pandas.Series)
            df=df[["pests","crops","description","input","output"]]
            df=pandas.concat([df.drop("description",axis=1),df['description'].apply(pandas.Series)],axis=1)
            df.rename(columns={'other':'description'}, inplace=True)
            df=df.drop(['created_by', 'age', 'assumptions', 'peer_review', 'case_studies'],axis=1)
            
            return df
        
        else:
            print(self.catalog)
    
    def get(self, dss="no.nibio.vips", model="PSILARTEMP"):
        """[Get model]

        Parameters
        ----------
        dss : str, optional
            [description], by default "no.nibio.vips"
        model : str, optional
            [description], by default "PSILARTEMP"
        """
        
        
        d={dss:[model for model in h.catalog[dss]] for dss in h.catalog} # dict with dss:model
        
        if (dss in d and model in d[dss]):
            return Model(dss,model)
        else:
            raise NotImplementedError()
        

class Model(Hub):
    """ Model Class derived from Hub. It allows to displays informations and run model and plot output

    Parameters
    ----------
    Hub : class
        Class allows to access IPM catalog and get one model
    """
    
    def __init__(self, dss, model):
        """Init of Model class model

        Parameters
        ----------
        dss : str
            id of the dss
        model : str
            id of the model
        """
        self.dss=dss
        self.model=model
        Hub.__init__(self)
    
    @property    
    def information(self):
        """ Return information of the model

        Returns
        -------
        dict
            dict containing model information 
        """
        return self.catalog[self.dss][self.model]
    
    def __xarray_convert__(self, output):
        """ Convert model output of the model in xarray dataset 

        Parameters
        ----------
        output : dict
            output of (run) model 

        Returns
        -------
        xarray.Dataset
            return a xarray.Dataset containing output of model with meta-information (attribut)
        """
        
        data = {str(var): vals for var, vals in zip(output['resultParameters'], zip(*output['locationResult'][0]['data']))}
        
        output_result = {key: data[key] for key in [item["id"] for item in self.information["output"]["result_parameters"]]}
        #data['warningStatus']=output['locationResult'][0]['warningStatus']
        
        times_index= pandas.date_range(start=output['timeStart'], end=output['timeEnd'], freq=str(output['interval'])+"s")
        
        df=pandas.DataFrame(output_result, index=times_index.values)
        
        # convert dataframe to xarray dataset
        ds = xr.Dataset.from_dataframe(df)
        ds = ds.rename({'index':'time'})
        
        # add attribut
        ds.time.attrs["name"]="time"
        ds.time.attrs["units"]="days"
        
        #variable attributs
        source = self.information

        data_vars_attrs={el['id']:{key:el[key] for key in ['title','description']} for el in source['output']['result_parameters']}

        for el in list(ds.data_vars):
            if el in list(data_vars_attrs):
                ds.data_vars[el].attrs=data_vars_attrs[el]
                
        # dataset attribut
        attrs={}
        attrs['name']=source['name']
        attrs['id']=source['id']
        attrs['version']=source['version']
        attrs['authors']=source['authors'][0]
        attrs['description']=source['description']["other"]
        attrs['description_url']=source['description_URL']

        ds.attrs = attrs
        
        return ds
    
    def run(self,weatherdata=None,fieldObservation=None,view="ds"):
        """ Run model

        Parameters
        ----------
        weatherdata : object, optional
            list of json containing weather data information from ipm weatherdata service, by default None
        fieldObservation : object, optional
            json object containing field observation, by default None
        view : str, optional
            parameter controlling the type of output (xarray.Dataset or json), by default "ds"
        """
        
        def input(weatherdata=None, fieldObservation=None):
            """ Fonction to create input data model according to IPM format

            Parameters
            ----------
            weatherdata : Object JSON, optional
                weatherdata from weather data ipm services in json format, by default None
            fieldObservation : Object JSON, optional
                field observation data in json format according IPM format, by default None

            Returns
            -------
            Object JSON
                return input json object to run model
            """
            
            if weatherdata:
                                                
                d= {"modelId": self.model,
                    "configParameters": {
                        "timeZone": "UTC",
                        "timeStart": weatherdata[0]["timeStart"],
                        "timeEnd": weatherdata[0]["timeStart"]
                        },
                    "weatherData": weatherdata[0]
                }
                return json.dumps(d)
            
            else:
                pass #fieldoservation file TODO
            
            
        output= self.ipm_hub.run_model(ModelId=self.dss,
            DSSId=self.model,
            model_input= input(weatherdata=weatherdata,fieldObservation=fieldObservation))
        
        if view== "ds":
            return self.__xarray_convert__(output=output)
        else:
            return output
    
    def plot(self,output):
        """Plot output result

        Parameters
        ----------
        output : xarray.Dataset
            Return a plot from output model conform to the description of output information of the model 
        """
        output.to_dataframe().plot.line()
        plt.ylabel(self.information["output"]['chart_heading'])
        plt.title(self.information["output"]['chart_heading'])
        plt.suptitle(output.name)  
        

        
        
    
        
         
        

In [20]:
from weatherdata.ipm import WeatherDataHub
ws=WeatherDataHub()
ws.list_resources()
fmi=ws.get_ressource('Finnish Meteorological Institute measured data')
weather=fmi.data(format="json")



INFO:start connecting to station 101104


In [103]:
h=Hub()

psi=h.get()
psi.run(weatherdata=weather,view='')

   

{'timeStart': '2020-06-12T00:00:00Z',
 'timeEnd': '2020-07-02T00:00:00Z',
 'interval': 86400,
 'resultParameters': ['TMDD5C',
  'THRESHOLD_1',
  'THRESHOLD_2',
  'TMD5C',
  'TMD',
  'THRESHOLD_3'],
 'locationResult': [{'longitude': None,
   'latitude': None,
   'altitude': None,
   'data': [[13.03, 260.0, 360.0, 13.03, 18.03, 560.0],
    [26.63, 260.0, 360.0, 13.61, 18.61, 560.0],
    [40.01, 260.0, 360.0, 13.37, 18.37, 560.0],
    [54.75, 260.0, 360.0, 14.74, 19.74, 560.0],
    [69.74, 260.0, 360.0, 14.99, 19.99, 560.0],
    [85.29, 260.0, 360.0, 15.55, 20.55, 560.0],
    [98.68, 260.0, 360.0, 13.39, 18.39, 560.0],
    [114.17, 260.0, 360.0, 15.48, 20.48, 560.0],
    [130.06, 260.0, 360.0, 15.9, 20.9, 560.0],
    [142.85, 260.0, 360.0, 12.78, 17.78, 560.0],
    [156.01, 260.0, 360.0, 13.16, 18.16, 560.0],
    [171.8, 260.0, 360.0, 15.79, 20.79, 560.0],
    [189.42, 260.0, 360.0, 17.62, 22.62, 560.0],
    [207.25, 260.0, 360.0, 17.83, 22.83, 560.0],
    [223.91, 260.0, 360.0, 16.65, 21

In [56]:

h=Hub()
#h.display()


result_dict

{'TMDD5C': (13.03,
  26.63,
  40.01,
  54.75,
  69.74,
  85.29,
  98.68,
  114.17,
  130.06,
  142.85,
  156.01,
  171.8,
  189.42,
  207.25,
  223.91,
  241.83,
  258.94,
  271.64,
  281.26,
  290.02,
  298.23),
 'THRESHOLD_1': (260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0,
  260.0),
 'THRESHOLD_2': (360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0,
  360.0),
 'THRESHOLD_3': (560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0,
  560.0)}

In [ ]:
h.catalog

In [ ]:
df=pandas.Series(h.dss_catalog).apply(pandas.Series).stack().apply(pandas.Series)

df=df[['name','pests', 'crops','platform_validated','description']]
df=pandas.concat([df.drop("description",axis=1),df['description'].apply(pandas.Series)],axis=1)
df.rename(columns={'other':'description'}, inplace=True)
df=df.drop(['created_by', 'age', 'assumptions', 'peer_review', 'case_studies'],axis=1)
df

In [ ]:
hub.list_dss(View)


## Access to one DSS

In [ ]:
#PSI=hub.get_dss(ModelId='no.nibio.vips', DSSId='PSILARTEMP') 
# hub[dss][model]
PSI=hub.get_model(dss="no.nibio.vips", model='PSILARTEMP')
print(PSI)# PSILARTEMP

In [ ]:
# 
weather_data = ...
PSI(weather_data)
PSI.input_DSS_weather_model_json(TimeStart="2021-01-01", 
                                 TimeEnd="2021-12-31",
                                 weatherDataService='Finnish Meteorological Institute measured data',
                                 parameters=[1002],
                                 stationId=[101104])

In [ ]:
ds=PSI.get_data_model(modelInput='model_input_weatherdata.json')
ds

In [ ]:
ds.to_dataframe().plot.line()

In [ ]:
PSILAROBS= hub.get_dss(DSSId='PSILAROBSE')


In [ ]:
PSILAROBS.input_DSS_fieldobservation_model_json(fieldobservation=r"C:\Users\mlabadie\Documents\GitHub\dss\src\openalea\dss\fieldobservation.json")

In [ ]:
PSILAROBS.get_data_model(modelInput="model_input_fieldobservation.json")